In [ ]:
import polars as pl
from util import mock_snakemake

if "snakemake" not in globals() or hasattr(snakemake, "mock"):  # noqa: F821
    snakemake = mock_snakemake("compute_capacity_factors")

VARIABLE_RENEWABLE_TYPES = ["wind", "solar"]

In [2]:
gen = pl.read_parquet(snakemake.input.gen_capacity)
df = pl.read_csv(snakemake.input.gen_dispatch, null_values=["NA", "#VALUE!"])
df = df.with_columns(pl.col("Date").str.to_datetime("%d-%m-%Y %H:%M"))
df = df.rename(lambda c: c.lower())
df

date,hour,interval,load,solar,wind,net load,renewables,nuclear,large hydro,imports,generation,thermal,load less (generation+imports)
datetime[μs],i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2019-01-01 00:00:00,1,1,22320.49429,0.0,2862.309099,19458.18519,4662.44597,2272.38118,2366.01883,6275.014973,16047.98977,6747.14379,-2.510453
2019-01-01 01:00:00,2,1,21527.97042,0.0,2515.56231,19012.40811,4282.111911,2273.184635,2384.25768,6232.057172,15298.45224,6358.898014,-2.538992
2019-01-01 02:00:00,3,1,20836.4152,0.0,2351.416739,18484.99846,4130.582703,2274.356083,2160.2123,6007.834256,14833.19079,6268.039704,-4.609846
2019-01-01 03:00:00,4,1,20422.81794,0.0,2445.068525,17977.74942,4215.448265,2274.105854,1963.69373,5886.635995,14539.40932,6086.161471,-3.227375
2019-01-01 04:00:00,5,1,20263.46628,0.0,2313.330378,17950.1359,4088.050291,2273.530573,1965.07719,5883.068751,14382.30385,6055.645796,-1.906321
…,…,…,…,…,…,…,…,…,…,…,…,…,…
2019-12-31 19:00:00,20,1,26376.32956,0.0,363.493413,26012.83615,1852.591697,2271.264335,3327.33283,10898.02667,15471.02736,8019.838498,7.27553
2019-12-31 20:00:00,21,1,25430.99307,0.0,353.855947,25077.13712,1838.574723,2271.717485,3232.241044,10287.77333,15137.381,7794.847748,5.83874
2019-12-31 21:00:00,22,1,24520.57146,0.0,273.717488,24246.85397,1759.807844,2271.308664,3051.20003,9798.533333,14718.52334,7636.206803,3.514787


In [5]:
# get large hydro upper limit
max_energy_genearation = pl.DataFrame(
    {"type": ["hydro"], "limit": [df.get_column("large hydro").sum()]}
)
max_energy_genearation.write_parquet(snakemake.output.yearly_limit)
max_energy_genearation

type,limit
str,f64
"""hydro""",2.6350e7


In [6]:
max_capacity = (
    gen.select("type", "Pmax")
    .group_by("type")
    .sum()
    .filter(pl.col("type").is_in(VARIABLE_RENEWABLE_TYPES))
)
max_capacity

type,Pmax
str,f64
"""wind""",5303.0
"""solar""",12795.400002


In [7]:
df2 = df.select(["date"] + VARIABLE_RENEWABLE_TYPES)
df2

date,wind,solar
datetime[μs],f64,f64
2019-01-01 00:00:00,2862.309099,0.0
2019-01-01 01:00:00,2515.56231,0.0
2019-01-01 02:00:00,2351.416739,0.0
2019-01-01 03:00:00,2445.068525,0.0
2019-01-01 04:00:00,2313.330378,0.0
…,…,…
2019-12-31 19:00:00,363.493413,0.0
2019-12-31 20:00:00,353.855947,0.0
2019-12-31 21:00:00,273.717488,0.0


In [16]:
df3 = df2
for gen_type in VARIABLE_RENEWABLE_TYPES:
    df3 = df3.with_columns(
        (
            pl.col(gen_type)
            / max_capacity.filter(pl.col("type") == gen_type).get_column("Pmax")
        )
    )
df3 = df3.unpivot(index=["date"], variable_name="type", value_name="capacity_factor")
df3

date,type,capacity_factor
datetime[μs],str,f64
2019-01-01 00:00:00,"""wind""",0.539753
2019-01-01 01:00:00,"""wind""",0.474366
2019-01-01 02:00:00,"""wind""",0.443413
2019-01-01 03:00:00,"""wind""",0.461073
2019-01-01 04:00:00,"""wind""",0.436231
…,…,…
2019-12-31 19:00:00,"""solar""",0.0
2019-12-31 20:00:00,"""solar""",0.0
2019-12-31 21:00:00,"""solar""",0.0


In [26]:
cf_by_hour = (
    df3.group_by(pl.col("date").dt.hour().alias("hour"), "type")
    .mean()
    .sort("hour")
    .drop("date")
)
cf_by_hour.plot.line(x="hour", y="capacity_factor", color="type")

alt.Chart(...)

In [27]:
cf_by_hour = (
    df3.group_by(pl.col("date").dt.month().alias("month"), "type")
    .mean()
    .sort("month")
    .drop("date")
)
cf_by_hour.plot.line(x="month", y="capacity_factor", color="type")

alt.Chart(...)

In [29]:
df3.write_parquet(snakemake.output.vcf)